Questo notebook permette, preso il csv delle recensioni di allenare LDA sul testo delle recensioni, opportunamente preprocessato e di ricavare per ogni recensione la dstribuzione in termini di Topic. Sulla base di questi risultati ho generato una nuova colonna nel dataset di partenza chiamata TopicDistribuition dove ho inserito per ogni recensione la corrispondente distribuzione di topic, ed ho effettuato un drop di tutte le altre colonne che non mi servivano. Alla fine ottengo un csv chiamato FinalDataset contenente soltanto le colonne review per le recensioni, category ovvero le nostre label e TopicDistribuition.

In [ ]:
import pprint
import nltk
import re
import os
import gensim
import pickle
from gensim import models
from gensim.models import LdaModel
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation,strip_numeric
from gensim.test.utils import datapath
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim import corpora
from textblob import Word
from textblob import TextBlob
from collections import defaultdict
from csv import reader
import pandas as pd
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation,strip_numeric
from textblob import TextBlob
from gensim.test.utils import datapath
 
# SNIPPET PER APRIRE IL CORPORA SERIALIZZATO
# Apro quanto serializzato
with open ('/content/drive/MyDrive/Colab Notebooks/Serialized_Processed_Corpora', 'rb') as fp:
    LISTA_CORPUS_PREPROCESSATI = pickle.load(fp)
df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reviewsincsvconExcel.csv",error_bad_lines=False, sep=';')
print(df)

#Creo un dizionario dal corpora, contenente il numero delle volte che una
#parola appare nel training set
dictionary= gensim.corpora.Dictionary(LISTA_CORPUS_PREPROCESSATI)
count = 0
for k, v in dictionary.iteritems():
        count +=1

#filtriamo i token che appaiono in meno di 15 documenti (numero assoluto) o
#in più di 0.5 documenti (frazione della taglia totale del corpora, non il numero
#assoluto). Dopo i primi due step, manteniamo soltanto i 600 token più frequenti
#600 è un parametro arbitrario, ho scelto di provare per questo per iniziare dato
#che sono state trovate circa 700 parole diverse e vorrei cercare di mantenerne
#il più possibile
dictionary.filter_extremes(no_below=15,no_above=0.5,keep_n=600)

#Per ogni documento creiamo un dizionario che riporta quante parole e
#quante volte queste appaiono. Salviamo questo nell'oggetto bow_corpus
bow_corpus = [dictionary.doc2bow(doc) for doc in LISTA_CORPUS_PREPROCESSATI]
 
# Train the model on the corpus.
# lda = LdaModel(bow_corpus,id2word=dictionary,passes=5, num_topics=8)
# temp_file = datapath("model")
# lda.save(temp_file)
lda = LdaModel.load(temp_file)
#mostro a schermo tutti i topic trovati con la relativa distribuzione
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

#Per ogni riga del csv. Prendo il testo della review, lo converto in BoW e lo do in pasto ad lda.get_document_topics
#E scrivo il risultato nella colonna TopicDistribuition. Una volta finito tutto salvo il tutto in un csv
def func(row):
  return lda.get_document_topics(dictionary.doc2bow(gensim.utils.tokenize(row.review)), minimum_probability=0.0)

df['TopicDistribuition']=df.apply(func, axis = 1)
print("\n")
print(df)
df.to_csv("/content/drive/MyDrive/Colab Notebooks/FinalDataset.csv",index=False)














                                                   review  ... TopicDistribuition
0       Great app! The new version now works on my Bra...  ...                NaN
1       Great It's not fully optimised and has some is...  ...                NaN
2       Works on a Nexus 6p I'm still messing around w...  ...                NaN
3       The bandwidth seemed to be limited to maximum ...  ...                NaN
4       Works well with my Hackrf Hopefully new update...  ...                NaN
...                                                   ...  ...                ...
288060  it doesn't do anything after installing this i...  ...                NaN
288061  I like this app . Its is very helpful for use....  ...                NaN
288062  Finally Brings back the Unix command line to A...  ...                NaN
288063  The API feature is great  just need loads more...  ...                NaN
288064  Works Nicely! I wish there were instructions t...  ...                NaN

[288065 rows x 